In [1]:
import os
import sys
import subprocess
import datetime as dt
import itertools
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster
import dask

In [2]:
cluster = LocalCluster(n_workers=4,threads_per_worker=1)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 0.97 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43743,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 0.97 TiB
Comm: tcp://127.0.0.1:37881,Total threads: 1
Dashboard: http://127.0.0.1:48243/status,Memory: 496.08 GiB
Nanny: tcp://127.0.0.1:34155,


In [3]:
workdir='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW_daily/'
mdirC5='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5'
fnameCanESMjoined=lambda mdir, yyyy, mm, dd, hh: f"{mdir}/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S{yyyy:04}{mm:02}{dd:02}{hh:02}_ocean_6hr_surface_tso.nc"
fnameCanESMdaily=lambda mdir, yyyy, mm, dd, hh: f"{mdir}/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S{yyyy:04}{mm:02}{dd:02}{hh:02}_ocean_1d_surface_tso.nc"
fnameCanESMAnom=lambda mdir, climyfirst,climylast,lfirst, llast, yyyy, mm: f"{mdir}/joined/test2_cwao_CanESM5.1p1bc-v20240611_hindcast_C{climyfirst:04}_{climylast:04}_SYr{yyyy:04}Mon{mm:02}_L_{lfirst:03}_{llast:03}_ocean_1d_surface_tso.nc"

In [4]:
climyrs=[1993,2023]
nlead=215

In [5]:
%%time
for mm in range(1,2):#13): # month loop
    print(f"Month:{mm} {dt.datetime.now()}",flush=True)
    for ix in range(0,5):#int(nlead/5)):
        fname=fnameCanESMAnom(mdirC5,climyrs[0],climyrs[-1],5*ix,5*(ix+1)-1,mm)
        if not os.path.exists(fname): # skip if file already exists
            flist=[fnameCanESMdaily(mdirC5,yyyy,mm,1,0) for yyyy in range(1993,2025) if yyyy<2024 or mm<=6] # stop at Jul 2024
            # chunks={'reftime':-1,'leadtime':5,'r':-1,'lat':-1,'lon':-1}
            with xr.open_mfdataset(flist,parallel=True,combine='nested',concat_dim='reftime',
                                   chunks={'reftime':-1,'leadtime':-1,'r':-1,'lat':-1,'lon':-1},
                                   preprocess=lambda ff: ff.isel(leadtime=slice(5*ix,5*(ix+1)))) as ff:
                #display(ff)
                EClim=ff.tso.sel(reftime=slice(np.datetime64(f'{climyrs[0]:04}-01-01'),np.datetime64(f'{climyrs[-1]:04}-12-31'))).mean(dim='r').mean(dim='reftime')
                Anom0=ff.tso-EClim
                for iy in range(1993,2025):
                    if iy<2024 or mm<=6: # stop at Jul 2024
                        Anomout
                        Anom0.to_netcdf(fname,mode='w')
                del EClim; del Anom0;

Month:1 2024-09-20 17:45:13.033782


<xarray.Dataset> Size: 2GB
Dimensions:   (reftime: 32, lat: 180, bnds: 2, lon: 360, r: 20, leadtime: 5)
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 40B 0 days 15:00:00 ... 4 days 15:00:00
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
    time      (reftime, leadtime) datetime64[ns] 1kB dask.array<chunksize=(1, 5), meta=np.ndarray>
Dimensions without coordinates: bnds, r
Data variables:
    lat_bnds  (reftime, lat, bnds) float64 92kB dask.array<chunksize=(1, 180, 2), meta=np.ndarray>
    lon_bnds  (reftime, lon, bnds) float64 184kB dask.array<chunksize=(1, 360, 2), meta=np.ndarray>
    tso       (reftime, r, leadtime, lat, lon) float64 2GB dask.array<chunksize=(1, 20, 5, 180, 360), meta=np.ndarray>
Attributes: (12/21)
    comment:                  Produced by ECCC (Environment and Climate Chang...
    summary:                  Seasonal Forecast data produced by ECCC as its ...
    title:                    ECCC seasonal forecast model output prepared fo...
    forecast_type:            hindcast
    Conventions:              CF-1.6 C3S-0.1
    project:                  C3S Seasonal Forecast
    ...                       ...
    forecast_reference_time:  1993-01-01T00:00:00Z
    history:                  Sat Aug 31 04:30:30 2024: ncecat /space/hall5/s...
    data_licence:             1) GRANT OF LICENCE - The Government of Canada ...
    commit:                    
    creation_date:            2024-07-07T07:30:39Z
    NCO:                      netCDF Operators version 5.0.5 (Homepage = http...

<xarray.Dataset> Size: 2GB
Dimensions:   (reftime: 32, lat: 180, bnds: 2, lon: 360, r: 20, leadtime: 5)
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 40B 5 days 15:00:00 ... 9 days 15:00:00
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
    time      (reftime, leadtime) datetime64[ns] 1kB dask.array<chunksize=(1, 5), meta=np.ndarray>
Dimensions without coordinates: bnds, r
Data variables:
    lat_bnds  (reftime, lat, bnds) float64 92kB dask.array<chunksize=(1, 180, 2), meta=np.ndarray>
    lon_bnds  (reftime, lon, bnds) float64 184kB dask.array<chunksize=(1, 360, 2), meta=np.ndarray>
    tso       (reftime, r, leadtime, lat, lon) float64 2GB dask.array<chunksize=(1, 20, 5, 180, 360), meta=np.ndarray>
Attributes: (12/21)
    comment:                  Produced by ECCC (Environment and Climate Chang...
    summary:                  Seasonal Forecast data produced by ECCC as its ...
    title:                    ECCC seasonal forecast model output prepared fo...
    forecast_type:            hindcast
    Conventions:              CF-1.6 C3S-0.1
    project:                  C3S Seasonal Forecast
    ...                       ...
    forecast_reference_time:  1993-01-01T00:00:00Z
    history:                  Sat Aug 31 04:30:30 2024: ncecat /space/hall5/s...
    data_licence:             1) GRANT OF LICENCE - The Government of Canada ...
    commit:                    
    creation_date:            2024-07-07T07:30:39Z
    NCO:                      netCDF Operators version 5.0.5 (Homepage = http...

<xarray.Dataset> Size: 2GB
Dimensions:   (reftime: 32, lat: 180, bnds: 2, lon: 360, r: 20, leadtime: 5)
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 40B 10 days 15:00:00 ... 14 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
    time      (reftime, leadtime) datetime64[ns] 1kB dask.array<chunksize=(1, 5), meta=np.ndarray>
Dimensions without coordinates: bnds, r
Data variables:
    lat_bnds  (reftime, lat, bnds) float64 92kB dask.array<chunksize=(1, 180, 2), meta=np.ndarray>
    lon_bnds  (reftime, lon, bnds) float64 184kB dask.array<chunksize=(1, 360, 2), meta=np.ndarray>
    tso       (reftime, r, leadtime, lat, lon) float64 2GB dask.array<chunksize=(1, 20, 5, 180, 360), meta=np.ndarray>
Attributes: (12/21)
    comment:                  Produced by ECCC (Environment and Climate Chang...
    summary:                  Seasonal Forecast data produced by ECCC as its ...
    title:                    ECCC seasonal forecast model output prepared fo...
    forecast_type:            hindcast
    Conventions:              CF-1.6 C3S-0.1
    project:                  C3S Seasonal Forecast
    ...                       ...
    forecast_reference_time:  1993-01-01T00:00:00Z
    history:                  Sat Aug 31 04:30:30 2024: ncecat /space/hall5/s...
    data_licence:             1) GRANT OF LICENCE - The Government of Canada ...
    commit:                    
    creation_date:            2024-07-07T07:30:39Z
    NCO:                      netCDF Operators version 5.0.5 (Homepage = http...

<xarray.Dataset> Size: 2GB
Dimensions:   (reftime: 32, lat: 180, bnds: 2, lon: 360, r: 20, leadtime: 5)
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 40B 15 days 15:00:00 ... 19 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
    time      (reftime, leadtime) datetime64[ns] 1kB dask.array<chunksize=(1, 5), meta=np.ndarray>
Dimensions without coordinates: bnds, r
Data variables:
    lat_bnds  (reftime, lat, bnds) float64 92kB dask.array<chunksize=(1, 180, 2), meta=np.ndarray>
    lon_bnds  (reftime, lon, bnds) float64 184kB dask.array<chunksize=(1, 360, 2), meta=np.ndarray>
    tso       (reftime, r, leadtime, lat, lon) float64 2GB dask.array<chunksize=(1, 20, 5, 180, 360), meta=np.ndarray>
Attributes: (12/21)
    comment:                  Produced by ECCC (Environment and Climate Chang...
    summary:                  Seasonal Forecast data produced by ECCC as its ...
    title:                    ECCC seasonal forecast model output prepared fo...
    forecast_type:            hindcast
    Conventions:              CF-1.6 C3S-0.1
    project:                  C3S Seasonal Forecast
    ...                       ...
    forecast_reference_time:  1993-01-01T00:00:00Z
    history:                  Sat Aug 31 04:30:30 2024: ncecat /space/hall5/s...
    data_licence:             1) GRANT OF LICENCE - The Government of Canada ...
    commit:                    
    creation_date:            2024-07-07T07:30:39Z
    NCO:                      netCDF Operators version 5.0.5 (Homepage = http...

<xarray.Dataset> Size: 2GB
Dimensions:   (reftime: 32, lat: 180, bnds: 2, lon: 360, r: 20, leadtime: 5)
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 40B 20 days 15:00:00 ... 24 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
    time      (reftime, leadtime) datetime64[ns] 1kB dask.array<chunksize=(1, 5), meta=np.ndarray>
Dimensions without coordinates: bnds, r
Data variables:
    lat_bnds  (reftime, lat, bnds) float64 92kB dask.array<chunksize=(1, 180, 2), meta=np.ndarray>
    lon_bnds  (reftime, lon, bnds) float64 184kB dask.array<chunksize=(1, 360, 2), meta=np.ndarray>
    tso       (reftime, r, leadtime, lat, lon) float64 2GB dask.array<chunksize=(1, 20, 5, 180, 360), meta=np.ndarray>
Attributes: (12/21)
    comment:                  Produced by ECCC (Environment and Climate Chang...
    summary:                  Seasonal Forecast data produced by ECCC as its ...
    title:                    ECCC seasonal forecast model output prepared fo...
    forecast_type:            hindcast
    Conventions:              CF-1.6 C3S-0.1
    project:                  C3S Seasonal Forecast
    ...                       ...
    forecast_reference_time:  1993-01-01T00:00:00Z
    history:                  Sat Aug 31 04:30:30 2024: ncecat /space/hall5/s...
    data_licence:             1) GRANT OF LICENCE - The Government of Canada ...
    commit:                    
    creation_date:            2024-07-07T07:30:39Z
    NCO:                      netCDF Operators version 5.0.5 (Homepage = http...

CPU times: user 22 s, sys: 2.24 s, total: 24.2 s
Wall time: 3min 42s


In [6]:
%%time
for mm in range(1,2):#13): # month loop
    print(f"Month:{mm} {dt.datetime.now()}",flush=True)
    for ix in range(5,10):#range(0,int(nlead/5)):
        fname=fnameCanESMAnom(mdirC5,climyrs[0],climyrs[-1],5*ix,5*(ix+1)-1,mm)
        if not os.path.exists(fname): # skip if file already exists
            flist=[fnameCanESMdaily(mdirC5,yyyy,mm,1,0) for yyyy in range(1993,2025) if yyyy<2024 or mm<=6] # stop at Jul 2024
            # chunks={'reftime':-1,'leadtime':5,'r':-1,'lat':-1,'lon':-1}
            with xr.open_mfdataset(flist,parallel=True,combine='nested',concat_dim='reftime',
                                   chunks={'reftime':-1,'leadtime':-1,'r':-1,'lat':-1,'lon':-1},
                                   preprocess=lambda ff: ff.isel(leadtime=slice(5*ix,5*(ix+1)))) as ff:
                ff2=ff.tso.chunk({'reftime':-1,'leadtime':-1,'r':-1,'lat':-1,'lon':-1})
                display(ff2)
                EClim=ff2.sel(reftime=slice(np.datetime64(f'{climyrs[0]:04}-01-01'),np.datetime64(f'{climyrs[-1]:04}-12-31'))).mean(dim='r').mean(dim='reftime')
                Anom0=ff2-EClim
                Anom0.to_netcdf(fname,mode='w')
                del EClim; del Anom0; del ff2;

Month:1 2024-09-20 17:48:56.009768


<xarray.DataArray 'tso' (reftime: 32, r: 20, leadtime: 5, lat: 180, lon: 360)> Size: 2GB
dask.array<rechunk-merge, shape=(32, 20, 5, 180, 360), dtype=float64, chunksize=(32, 20, 5, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 40B 25 days 15:00:00 ... 29 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
    time      (reftime, leadtime) datetime64[ns] 1kB dask.array<chunksize=(32, 5), meta=np.ndarray>
Dimensions without coordinates: r
Attributes:
    cell_methods:   leadtime: point
    grid_mapping:   hcrs
    long_name:      Sea Surface Temperature
    standard_name:  sea_surface_temperature
    units:          K

<xarray.DataArray 'tso' (reftime: 32, r: 20, leadtime: 5, lat: 180, lon: 360)> Size: 2GB
dask.array<rechunk-merge, shape=(32, 20, 5, 180, 360), dtype=float64, chunksize=(32, 20, 5, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 40B 30 days 15:00:00 ... 34 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
    time      (reftime, leadtime) datetime64[ns] 1kB dask.array<chunksize=(32, 5), meta=np.ndarray>
Dimensions without coordinates: r
Attributes:
    cell_methods:   leadtime: point
    grid_mapping:   hcrs
    long_name:      Sea Surface Temperature
    standard_name:  sea_surface_temperature
    units:          K

<xarray.DataArray 'tso' (reftime: 32, r: 20, leadtime: 5, lat: 180, lon: 360)> Size: 2GB
dask.array<rechunk-merge, shape=(32, 20, 5, 180, 360), dtype=float64, chunksize=(32, 20, 5, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 40B 35 days 15:00:00 ... 39 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
    time      (reftime, leadtime) datetime64[ns] 1kB dask.array<chunksize=(32, 5), meta=np.ndarray>
Dimensions without coordinates: r
Attributes:
    cell_methods:   leadtime: point
    grid_mapping:   hcrs
    long_name:      Sea Surface Temperature
    standard_name:  sea_surface_temperature
    units:          K

<xarray.DataArray 'tso' (reftime: 32, r: 20, leadtime: 5, lat: 180, lon: 360)> Size: 2GB
dask.array<rechunk-merge, shape=(32, 20, 5, 180, 360), dtype=float64, chunksize=(32, 20, 5, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 40B 40 days 15:00:00 ... 44 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
    time      (reftime, leadtime) datetime64[ns] 1kB dask.array<chunksize=(32, 5), meta=np.ndarray>
Dimensions without coordinates: r
Attributes:
    cell_methods:   leadtime: point
    grid_mapping:   hcrs
    long_name:      Sea Surface Temperature
    standard_name:  sea_surface_temperature
    units:          K

<xarray.DataArray 'tso' (reftime: 32, r: 20, leadtime: 5, lat: 180, lon: 360)> Size: 2GB
dask.array<rechunk-merge, shape=(32, 20, 5, 180, 360), dtype=float64, chunksize=(32, 20, 5, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 40B 45 days 15:00:00 ... 49 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
    time      (reftime, leadtime) datetime64[ns] 1kB dask.array<chunksize=(32, 5), meta=np.ndarray>
Dimensions without coordinates: r
Attributes:
    cell_methods:   leadtime: point
    grid_mapping:   hcrs
    long_name:      Sea Surface Temperature
    standard_name:  sea_surface_temperature
    units:          K

CPU times: user 17.7 s, sys: 1.82 s, total: 19.5 s
Wall time: 3min 11s


In [7]:
client.close()
cluster.close()

In [8]:
#1:51,1:35 with 4 workers
#1:43,1:40 with 6 workers
#3:42,3:11  with 2 workers
#Note: threads_per_worker=1 did not slow down code execution